In [1]:
import numpy as np

In [2]:
from classes.model import Model
from classes.scenario import Scenario
from classes.scenario_setup import Scenario_simpy
from classes.combine import Combine

In [3]:
units_to_include = [
    'GL13NN',
    'SN36BB',
    'BS105NB',
    'EX314JB',
    'PL68DH',
    'TR13LQ',
    'EX25DW',
    'BA13NG',
    'SP28BJ',
    'TA15DA',
    'TQ27AA',
    'BS234TQ',
    'BA214AT',
]

In [4]:
merge_dicts = dict(
    df_selected_units = {},
    df_selected_transfer = {},
    df_selected_lsoa = {},
    results_summary_by_admitting_unit = {},
    results_summary_by_lsoa = {}
)

## Scenario 1

In [5]:
scenario_name = 'drip-and-ship'

In [6]:
# Scenario overwrites default values
scenario = Scenario({
    # "setup": setup,
    'name': scenario_name,
    "run_duration": 5,  # years
})

df_units = scenario.get_unit_services()

df_units = df_units.reset_index()

mask = df_units['postcode'].isin(units_to_include)
df_units.loc[mask, 'selected'] = 1

df_units = df_units.set_index('postcode')

In [7]:
df_units[df_units['selected'] == 1]

,stroke_team,short_code,ssnap_name,use_ivt,use_mt,use_msu,transfer_unit_postcode,lsoa_code,region,region_code,region_type,country,icb,icb_code,isdn,selected
postcode,,,,,,,,,,,,,,,,
BA13NG,Royal United Hospital Bath,BA,Royal United Hospital Bath,1,0,0,nearest,E01014428,"NHS Bath and North East Somerset, Swindon and ...",E38000231,SICBL,England,"NHS Bath and North East Somerset, Swindon and ...",E54000040,"Gloucester, BSW, BNSSG and Somerset",1
EX314JB,"North Devon District Hospital, Barnstaple",BP,North Devon District Hospital,1,0,0,nearest,E01020134,NHS Devon ICB - 15N,E38000230,SICBL,England,NHS Devon Integrated Care Board,E54000037,"Devon, Cornwall and Isles of Scilly",1
PL68DH,"Derriford Hospital, Plymouth",DF,Derriford Hospital,1,1,1,nearest,E01015092,NHS Devon ICB - 15N,E38000230,SICBL,England,NHS Devon Integrated Care Board,E54000037,"Devon, Cornwall and Isles of Scilly",1
EX25DW,Royal Devon and Exeter Hospital,EX,Royal Devon and Exeter Hospital,1,0,0,nearest,E01020013,NHS Devon ICB - 15N,E38000230,SICBL,England,NHS Devon Integrated Care Board,E54000037,"Devon, Cornwall and Isles of Scilly",1
GL13NN,"Gloucestershire Royal Hospital, Gloucester",GL,Gloucestershire Royal Hospital,1,0,0,nearest,E01022311,NHS Gloucestershire ICB - 11M,E38000062,SICBL,England,NHS Gloucestershire Integrated Care Board,E54000043,"Gloucester, BSW, BNSSG and Somerset",1
SN36BB,"Great Western Hospital, Swindon",GW,Great Western Hospital Swindon,1,0,0,nearest,E01015583,"NHS Bath and North East Somerset, Swindon and ...",E38000231,SICBL,England,"NHS Bath and North East Somerset, Swindon and ...",E54000040,"Gloucester, BSW, BNSSG and Somerset",1
TA15DA,"Musgrove Park Hospital, Taunton",MP,Musgrove Park Hospital,1,0,0,nearest,E01029302,NHS Somerset ICB - 11X,E38000150,SICBL,England,NHS Somerset Integrated Care Board,E54000038,"Gloucester, BSW, BNSSG and Somerset",1
BS105NB,North Bristol Hospital (Southmead),NB,North Bristol Hospitals,1,1,1,nearest,E01014631,"NHS Bristol, North Somerset and South Gloucest...",E38000222,SICBL,England,"NHS Bristol, North Somerset and South Gloucest...",E54000039,"Gloucester, BSW, BNSSG and Somerset",1
SP28BJ,Salisbury District Hospital,SS,Salisbury District Hospital,1,0,0,nearest,E01032000,"NHS Bath and North East Somerset, Swindon and ...",E38000231,SICBL,England,"NHS Bath and North East Somerset, Swindon and ...",E54000040,"Gloucester, BSW, BNSSG and Somerset",1


In [8]:
dict_useful_data = scenario.process_scenario(df_units)

scenario_simpy = Scenario_simpy(
    dict_useful_data['df_units'],
    dict_useful_data['df_transfer'],
    dict_useful_data['df_lsoa'],
)

model = Model(scenario_simpy)

model.run()

In [9]:
# Update big dictionary of Dataframes:
merge_dicts['df_selected_units'][scenario_name] = scenario_simpy.df_selected_units
merge_dicts['df_selected_transfer'][scenario_name] = scenario_simpy.df_selected_transfer
merge_dicts['df_selected_lsoa'][scenario_name] = scenario_simpy.df_selected_lsoa
merge_dicts['results_summary_by_admitting_unit'][scenario_name] = model.results_summary_by_admitting_unit
merge_dicts['results_summary_by_lsoa'][scenario_name] = model.results_summary_by_lsoa

## Scenario 2

In [10]:
scenario_name = 'mothership'

In [11]:
# Scenario overwrites default values
scenario = Scenario({
    # "setup": setup,
    'name': scenario_name,
    "run_duration": 5,  # years
})

df_units = scenario.get_unit_services()

df_units = df_units.reset_index()

# Set all non-MT units to have no IVT:
mask = df_units['use_mt'] == 0
df_units.loc[mask, 'use_ivt'] = 0

mask = (
    (df_units['postcode'].isin(units_to_include)) &
    (df_units['use_ivt'] == 1)
)
df_units.loc[mask, 'selected'] = 1


df_units = df_units.set_index('postcode')

In [12]:
df_units[df_units['selected'] == 1]

,stroke_team,short_code,ssnap_name,use_ivt,use_mt,use_msu,transfer_unit_postcode,lsoa_code,region,region_code,region_type,country,icb,icb_code,isdn,selected
postcode,,,,,,,,,,,,,,,,
PL68DH,"Derriford Hospital, Plymouth",DF,Derriford Hospital,1,1,1,nearest,E01015092,NHS Devon ICB - 15N,E38000230,SICBL,England,NHS Devon Integrated Care Board,E54000037,"Devon, Cornwall and Isles of Scilly",1
BS105NB,North Bristol Hospital (Southmead),NB,North Bristol Hospitals,1,1,1,nearest,E01014631,"NHS Bristol, North Somerset and South Gloucest...",E38000222,SICBL,England,"NHS Bristol, North Somerset and South Gloucest...",E54000039,"Gloucester, BSW, BNSSG and Somerset",1


In [13]:
dict_useful_data = scenario.process_scenario(df_units)

scenario_simpy = Scenario_simpy(
    dict_useful_data['df_units'],
    dict_useful_data['df_transfer'],
    dict_useful_data['df_lsoa'],
)

model = Model(scenario_simpy)

model.run()

In [14]:
# Update big dictionary of Dataframes:
merge_dicts['df_selected_units'][scenario_name] = scenario_simpy.df_selected_units
merge_dicts['df_selected_transfer'][scenario_name] = scenario_simpy.df_selected_transfer
merge_dicts['df_selected_lsoa'][scenario_name] = scenario_simpy.df_selected_lsoa
merge_dicts['results_summary_by_admitting_unit'][scenario_name] = model.results_summary_by_admitting_unit
merge_dicts['results_summary_by_lsoa'][scenario_name] = model.results_summary_by_lsoa

## Combine multiple scenarios

In [15]:
combine = Combine()

combo_df_selected_units = combine.combine_selected_units(
    merge_dicts['df_selected_units'])

combo_df_selected_transfer = combine.combine_selected_transfer(
    merge_dicts['df_selected_transfer'])

combo_df_selected_lsoa = combine.combine_selected_lsoa(
    merge_dicts['df_selected_lsoa'])

combo_results_summary_by_admitting_unit = combine.combine_results_summary_by_admitting_unit(
    merge_dicts['results_summary_by_admitting_unit'])

combo_results_summary_by_lsoa = combine.combine_results_summary_by_lsoa(
    merge_dicts['results_summary_by_lsoa'])

In [16]:
combo_df_selected_units.T

postcode                                                                         BA13NG  \
scenario      property                                                                    
any           stroke_team                                    Royal United Hospital Bath   
              short_code                                                             BA   
              ssnap_name                                     Royal United Hospital Bath   
              lsoa_code                                                       E01014428   
              region                  NHS Bath and North East Somerset, Swindon and ...   
              region_code                                                     E38000231   
              region_type                                                         SICBL   
              country                                                           England   
              icb                     NHS Bath and North East Somerset, Swindon and ...   
              icb_code                                                        E54000040   
              isdn                                  Gloucester, BSW, BNSSG and Somerset   
drip-and-ship use_ivt                                                                 1   
              use_mt                                                                  0   
              use_msu                                                                 0   
              selected                                                                1   
              transfer_unit_postcode                                            nearest   
mothership    use_ivt                                                              <NA>   
              use_mt                                                               <NA>   
              use_msu                                                              <NA>   
              selected                                                             <NA>   
              transfer_unit_postcode                                               <NA>   

postcode                                                          BA214AT  \
scenario      property                                                      
any           stroke_team                        Yeovil District Hospital   
              short_code                                               YE   
              ssnap_name                         Yeovil District Hospital   
              lsoa_code                                         E01029231   
              region                               NHS Somerset ICB - 11X   
              region_code                                       E38000150   
              region_type                                           SICBL   
              country                                             England   
              icb                      NHS Somerset Integrated Care Board   
              icb_code                                          E54000038   
              isdn                    Gloucester, BSW, BNSSG and Somerset   
drip-and-ship use_ivt                                                   1   
              use_mt                                                    0   
              use_msu                                                   0   
              selected                                                  1   
              transfer_unit_postcode                              nearest   
mothership    use_ivt                                                <NA>   
              use_mt                                                 <NA>   
              use_msu                                                <NA>   
              selected                                               <NA>   
              transfer_unit_postcode                                 <NA>   

postcode                                                                        BS105NB  \
scenario      property                     

In [17]:
combo_df_selected_transfer.T

postcode                                 BA13NG BA214AT BS105NB BS234TQ  \
transfer_unit_postcode                  BS105NB BS105NB BS105NB BS105NB   
scenario      property                                                    
any           transfer_unit_travel_time    33.7    74.5     0.0    43.4   
              selected                        1       1       1       1   
drip-and-ship Use                             1       1       1       1   
mothership    Use                          <NA>    <NA>       1    <NA>   

postcode                                EX25DW EX314JB  GL13NN PL68DH  SN36BB  \
transfer_unit_postcode                  PL68DH  PL68DH BS105NB PL68DH BS105NB   
scenario      property                                                          
any           transfer_unit_travel_time   59.5    90.6    46.6    0.0    49.8   
              selected                       1       1       1      1       1   
drip-and-ship Use                            1       1       1      1       1   
mothership    Use                         <NA>    <NA>    <NA>      1    <NA>   

postcode                                 SP28BJ  TA15DA TQ27AA TR13LQ  
transfer_unit_postcode                  SO166YD BS105NB PL68DH PL68DH  
scenario      property                                                 
any           transfer_unit_travel_time    35.9    65.9   52.0   75.6  
              selected                        1       1      1      1  
drip-and-ship Use                             1       1      1      1  
mothership    Use                          <NA>    <NA>   <NA>   <NA>

In [18]:
combo_df_selected_lsoa.T

LSOA                             Basingstoke and Deane 003C  \
lsoa_code                                         E01022499   
scenario      property                                        
any           unit_travel_time                         35.9   
              selected                                    1   
              admissions                           1.666667   
              relative_frequency                   0.000237   
drip-and-ship unit_postcode                          SN36BB   
              Use                                         1   
mothership    unit_postcode                            <NA>   
              Use                                      <NA>   

LSOA                             Bath and North East Somerset 001B  \
lsoa_code                                                E01014400   
scenario      property                                               
any           unit_travel_time                                17.6   
              selected                                           1   
              admissions                                       4.0   
              relative_frequency                          0.000568   
drip-and-ship unit_postcode                                 BA13NG   
              Use                                                1   
mothership    unit_postcode                                   <NA>   
              Use                                             <NA>   

LSOA                             Bath and North East Somerset 001C  \
lsoa_code                                                E01014401   
scenario      property                                               
any           unit_travel_time                                18.7   
              selected                                           1   
              admissions                                  2.333333   
              relative_frequency                          0.000331   
drip-and-ship unit_postcode                                 BA13NG   
              Use                                                1   
mothership    unit_postcode                                   <NA>   
              Use                                             <NA>   

LSOA                             Bath and North East Somerset 002B  \
lsoa_code                                                E01014403   
scenario      property                                               
any           unit_travel_time                                18.7   
              selected                                           1   
              admissions                                  5.666667   
              relative_frequency                          0.000804   
drip-and-ship unit_postcode                                 BA13NG   
              Use                                                1   
mothership    unit_postcode                                   <NA>   
              Use                                             <NA>   

LSOA                             Bath and North East Somerset 002C  \
lsoa_code                                                E01014404   
scenario      property                                               
any           unit_travel_time                                19.8   
              selected                                           1   
              admissions                                  1.333333   
              relative_frequency                          0.000189   
drip-and-ship unit_postcode                                 BA13NG   
              Use                                                1   
mothership    unit_postcode                                   <NA>   
              Use                                             <NA>   

LSOA                             Bath and North East Somerset 003A  \
lsoa_code                                                E01014395   
scenario      property                                               
any         

In [19]:
combo_results_summary_by_admitting_unit.T

unit                                                                         BA13NG  \
scenario                            property                    subtype               
drip-and-ship                       time_ambulance_called       mean             30   
                                                                std               0   
                                    time_ambulance_arrival      mean             60   
                                                                std               0   
                                    time_ambulance_leaves_scene mean             80   
                                                                std               0   
                                    time_admitting_unit_arrival mean     105.651372   
                                                                std        8.737673   
                                    time_needle                 mean     135.243081   
                                                                std        8.989671   
                                    time_transfer_unit_arrival  mean     199.309346   
                                                                std        8.769335   
                                    time_puncture               mean     259.309346   
                                                                std        8.769335   
                                    mRS shift                   mean      -0.530559   
                                                                std        0.024173   
                                    utility_shift               mean       0.100551   
                                                                std         0.00755   
                                    mRS 0-2                     mean       0.600349   
                                                                std        0.136579   
mothership                          time_ambulance_called       mean           <NA>   
                                                                std            <NA>   
                                    time_ambulance_arrival      mean           <NA>   
                                                                std            <NA>   
                                    time_ambulance_leaves_scene mean           <NA>   
                                                                std            <NA>   
                                    time_admitting_unit_arrival mean           <NA>   
                                                                std            <NA>   
                                    time_needle                 mean           <NA>   
                                                                std            <NA>   
                                    time_transfer_unit_arrival  mean           <NA>   
                                                                std            <NA>   
                                    time_puncture               mean           <NA>   
                                                                std            <NA>   
                                    mRS shift                   mean           <NA>   
                                                                std            <NA>   
                                    utility_shift               mean           <NA>   
                                                                std            <NA>   
                                    mRS 0-2                     mean           <NA>   
                                                                std            <NA>   
diff_drip-and-ship_minus_mothership utility_shift               mean           <NA>   
                                                                std         0.00755   
                                    mRS shift                   mean           <NA>   
                                                                std        0.024173  

In [20]:
combo_results_summary_by_lsoa.T

lsoa                                                                    Basingstoke and Deane 003C  \
LSOA11CD                                                                                 E01022499   
scenario                            property                    subtype                              
drip-and-ship                       time_ambulance_called       mean                            30   
                                                                std                              0   
                                    time_ambulance_arrival      mean                            60   
                                                                std                              0   
                                    time_ambulance_leaves_scene mean                            80   
                                                                std                              0   
                                    time_admitting_unit_arrival mean                         115.9   
                                                                std                              0   
                                    time_needle                 mean                         145.9   
                                                                std                              0   
                                    time_transfer_unit_arrival  mean                          <NA>   
                                                                std                           <NA>   
                                    time_puncture               mean                          <NA>   
                                                                std                           <NA>   
                                    mRS shift                   mean                     -0.501154   
                                                                std                            0.0   
                                    utility_shift               mean                      0.092466   
                                                                std                            0.0   
                                    mRS 0-2                     mean                       0.68077   
                                                                std                            0.0   
mothership                          time_ambulance_called       mean                          <NA>   
                                                                std                           <NA>   
                                    time_ambulance_arrival      mean                          <NA>   
                                                                std                           <NA>   
                                    time_ambulance_leaves_scene mean                          <NA>   
                                                                std                           <NA>   
                                    time_admitting_unit_arrival mean                          <NA>   
                                                                std                           <NA>   
                                    time_needle                 mean                          <NA>   
                                                                std                           <NA>   
                                    time_transfer_unit_arrival  mean                          <NA>   
                                                                std                           <NA>   
                                    time_puncture               mean                          <NA>   
                                                                std                           <NA>   
                                    mRS shift                   mean                          <NA>   
                                                                std                           <NA>   
                      